In [15]:
import struct
import numpy as np

files = ["fashion-mnist_test.csv", "fashion-mnist_train.csv"]
for file in files:
    xs, ys = [], []
    with open(file, "r") as f:
        data = f.read()
    data = data.split("\n")[1:-1]
    for line in data:
        label, *pixels = line.split(",")
        xs.append([float(int(v)) / 255.0 for v in pixels])
        ys.append(int(label))
        
    xs = np.array(xs).astype(np.float32) 
    ys = np.array(ys).astype(np.uint8)
    
    xs = xs.tobytes()
    ys = ys.tobytes()
    
    with open(file.split(".")[0] + "-data.bin", "wb") as f:
        f.write(xs)
    with open(file.split(".")[0] + "-labels.bin", "wb") as f:
        f.write(ys)

In [18]:
import torch
n = torch.arange(30)
n = n.reshape((5,3,2))
print(n.shape)
print(n.flatten())
n = n.permute((0,1,2))
print(n[1][1][2])
print(n.shape)
n.flatten()

torch.Size([5, 3, 2])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])
tensor(15)
torch.Size([2, 3, 5])


tensor([ 0,  6, 12, 18, 24,  2,  8, 14, 20, 26,  4, 10, 16, 22, 28,  1,  7, 13,
        19, 25,  3,  9, 15, 21, 27,  5, 11, 17, 23, 29])

In [26]:
perm = (3,4,5,1,2,0)
for i in range(len(perm)):
    j = perm[i]
    while True:
        if j == i:
            break
        if j < i:
            print(i, j)
        j = perm[j]
    

1 0
2 0
2 1
3 1
3 2
3 0
4 2
4 0
4 3
4 1
5 0
5 3
5 1
5 4
5 2


In [65]:
import torchvision

dataset = torchvision.datasets.FashionMNIST(".", train=True, download=True, transform=torchvision.transforms.ToTensor())
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=64,
    shuffle=True,
    drop_last=True
)
model = torch.nn.Sequential(
    torch.nn.Linear(in_features=784, out_features=256),
    torch.nn.Tanh(),
    torch.nn.Linear(in_features=256, out_features=10),
)
optim = torch.optim.AdamW(model.parameters())
accs = []
for x, y in data_loader:
    x = x.view((-1, 784))
    y_ = model(x)
    loss = torch.nn.functional.cross_entropy(y_, y)
    optim.zero_grad()
    loss.backward()
    optim.step()
    acc = (y_.argmax(dim=-1) == y).float().mean().item()
    accs.append(acc)
accs = torch.tensor(accs)
print((accs[-20:]).mean())

# 3x3
# -> 1x9, 9x1, 3x3
# .view(-1, 3)
# shape 1,9
# .view(-1, 3)
# 3,3

# .shape(3,3)
# .view(-1, 9)
# 1,9

# 5,3,2 - 30
# .shape 1,30 2,15, 3,10, 5,6
# 5,3,2

# .view(-1, 2) -> 15,2

tensor(0.8562)


In [68]:
import numpy as np
# layer1_weights = model[0].weight
# layer2_bias = model[0].bias
models = [
    (model[0].weight.data, model[0].bias.data, "layer1"),
    (model[2].weight.data, model[2].bias.data, "layer2"),
]
for weight, bias, layer in models:
    xs = weight.numpy()
    ys = bias.numpy()
    
    xs = xs.tobytes()
    ys = ys.tobytes()
    
    with open(layer + "-weight.bin", "wb") as f:
        f.write(xs)
    with open(layer + "-bias.bin", "wb") as f:
        f.write(ys)

In [67]:
model[0].weight.data

tensor([[ 0.0137,  0.0232,  0.0011,  ..., -0.0028,  0.0309, -0.0142],
        [-0.0485,  0.0315,  0.0241,  ...,  0.0446,  0.0095, -0.0021],
        [ 0.0580, -0.0060, -0.0329,  ...,  0.0167,  0.0457, -0.0338],
        ...,
        [-0.0170, -0.0487,  0.0061,  ...,  0.0569,  0.1067,  0.0234],
        [-0.0432, -0.0104,  0.0126,  ..., -0.0089, -0.0302, -0.0473],
        [-0.0160, -0.0121, -0.0149,  ...,  0.0219, -0.0295,  0.0085]])